In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/kaggle/input/bank-marketing-dataset/bank.csv")

df.head()

In [ ]:
df["pdays"].describe()

All the percentiles are -1 and mean also seems to be less compare to max values 

In [ ]:
df["pdays"].value_counts()

-1 is the missing values, so we can consider removing it when describing. As many value are missing we cannot remove it 

In [ ]:
df["pdays"][df["pdays"]>0].describe()

Now we can see that the mean, median are acceptable value and also as mean is greater than median we can say that it is right skewed and therefore we should standardardize it

In [ ]:
sns.barplot(x="education",y="balance", data=df[df["education"]!="unknown"],estimator=np.median);

we can say that tertiary has the highest median compare to others

In [ ]:
plt.figure(figsize=(25,8))
sns.boxplot("pdays", data= df[df["pdays"]>0]);

In [ ]:
df.drop(df[df["pdays"]>600].index,inplace=True) # remove putliers

In [ ]:
df.head()

In [ ]:
df['deposit'].replace('no',0, inplace=True)
df['deposit'].replace('yes',1, inplace=True)

In [ ]:
df[df["pdays"]<0]

I will keep it as it is because if i change that to median and mean it will create a biasing like -1 as almost 90% data is missing 

In [ ]:
sns.barplot(x="deposit", y="age", data=df);

There is not much change w.r.t age

In [ ]:
sns.barplot(x="deposit", y="balance", data=df);

we can see that response are more if the average balance of a person is more

In [ ]:
sns.barplot(x="deposit", y="duration", data=df);

if a person talk more to a sales person at an average there is a high chance of him being converted

In [ ]:
sns.barplot(x="deposit", y="pdays", data=df[df['pdays']>0]);

we can see that there is high chance of a person to not response if a person is not frequently contacted

In [ ]:
sns.barplot(x="deposit", y="previous", data=df[df['previous']>0]);

In [ ]:
df.corr()

There is not much difference w.r.t response

## Data Preprocessing

In [ ]:
numeric_features_names = ['age','balance', 'day', 'duration','campaign', 'pdays', 'previous']
categorical_features_name = ['job','marital','education','default','housing','loan','contact','month', 'poutcome']

In [ ]:
from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
ss.fit(df[numeric_features_names])
df[numeric_features_names] = ss.transform(df[numeric_features_names])
#print(df)

In [ ]:
df = pd.get_dummies(df, columns=categorical_features_name,drop_first=True) #drop first = true to take care of dummy
#variable trap

#print(df)

In [ ]:
df.columns

In [ ]:
X = df[['age', 'balance', 'duration', 'campaign', 'pdays', 'previous',
        'job_blue-collar', 'job_housemaid',
       'job_management', 'job_retired', 'job_services',
       'job_student', 'job_technician', 'job_unemployed',
       'marital_married', 'marital_single', 'education_secondary',
       'education_tertiary', 'education_unknown', 'housing_yes',
       'loan_yes', 'contact_telephone', 'contact_unknown',
       'month_dec', 'month_feb', 'month_jan', 'month_jun',
       'month_mar', 'month_may', 'month_oct', 'month_sep',
       'poutcome_other', 'poutcome_success', 'poutcome_unknown'] ]
y = df["deposit"]

#### 'default_yes' , 'job_unknown', 'month_nov', 'month_aug', , 'job_entrepreneur' , 'job_self-employed',  'month_jul' , 'day' as been removed as the p-value of the variables are greater than 0.05 which means they are not significant

In [ ]:
import statsmodels.api as sm
OLS= sm.OLS(y,X).fit()
OLS.summary()

## Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
model = lr.fit(X_train,np.array(y_train))

print(model)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train, y_train, cv=5)
np.mean(scores)

mean of kfold accuracy is 81% let's check f1 score, precission and recall of dependent variable 

In [ ]:
from sklearn.metrics import classification_report
print("Classification Report:\n ", classification_report(y_test, y_pred))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=45,random_state=0,max_depth=30)
classifier=classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(classifier, X_train, y_train, cv=5)
np.mean(scores)

mean of kfold accuracy is 83% let's check f1 score, precission and recall of dependent variable 

In [ ]:
from sklearn.metrics import classification_report
print("Classification Report:\n ", classification_report(y_test, y_pred))

f1 score of 1 has also been increased from 81% to 82% so as the accuracy from 81% to 83%

### So we will prefer Random Forest rather than Logistics regression